Вы аналитик в команде монетизации классифайда. Ваша команда провела тест с экранами платных услуг в монетизации. Основная гипотеза заключалась в том, что мы сможем увеличить ARPU на +3.5 рубля.

У нас есть данные по продавцам авто, участвующим в тесте:

Будем анализировать этот тест.

Схема данных:
- `ARPU`: целевая метрика в тесте
- `is_capital`: продавец авто из столицы (capital) или из регионов (region)
- `is_pro`: профессиональный или нет продавец (1 — профи, 0 — простой)
- `group`: группа в А/Б-тесте (1 — тестовая, 0 — контрольная)

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.power import tt_ind_solve_power

### Задание 1
Проведите базовый t-test и сделайте вывод по его результатам.

Введите значение p-value в поле ниже с точностью до трех знаков в десятичной части. Используйте точку для разделения целой и дробной части.

In [2]:
df = pd.read_csv("stratification_homework_2.csv")

In [3]:
df.head()

,ARPU,is_capital,is_pro,group
0,250.5,region,1,1
1,182.0,capital,0,1
2,75.0,region,0,0
3,532.5,capital,1,0
4,88.0,region,0,1


In [4]:
group_a = df[df["group"] == 0]
group_b = df[df["group"] == 1]

In [5]:
_, pval = stats.ttest_ind(group_a["ARPU"], group_b["ARPU"])

In [6]:
round(pval, 3)

np.float64(0.065)

### Задание 2 (1/3)
Найдите наилучшую для разбиения страту.

Рассчитайте значение стратифицированной дисперсии для страты `is_capital` и введите его в поле ниже в формате целого числа.

In [7]:
def calculate_strat_var(data, strata_name, gen_pop_weights, target_value):
    """Вычисляет стратифицированную дисперсию."""

    strat_vars = data.groupby(strata_name)[target_value].var()

    data_vars_weights = pd.merge(
        pd.Series(strat_vars, name="value_vars"),
        pd.Series(gen_pop_weights, name="weight"),
        how="inner",
        left_index=True,
        right_index=True,
    )

    var_strat = (
        data_vars_weights["weight"] * data_vars_weights["value_vars"]
    ).sum()
    return var_strat

In [8]:
df["ARPU"].var()

np.float64(7480.429364766055)

In [9]:
capital_wght = len(df[df["is_capital"] == "capital"]) / len(df)

round(
    calculate_strat_var(
        df,
        "is_capital",
        {"capital": capital_wght, "region": (1 - capital_wght)},
        "ARPU",
    )
)

5790

### Задание 2 (2/3)

Рассчитайте значение стратифицированной дисперсии для страты `is_pro` и введите его в поле ниже в формате целого числа.

In [10]:
not_pro_wght = len(df[df["is_pro"] == 0]) / len(df)

round(
    calculate_strat_var(
        df,
        "is_pro",
        {0: not_pro_wght, 1: (1 - not_pro_wght)},
        "ARPU",
    )
)

6194

### Задание 2 (3/3)
Укажите страту, при которой дисперсия сокращается максимально.

**Ответ:** `is_capital`

### Задание 3
Используя наилучшую страту, примените постстратификацию к данным и пересчитайте результаты t-test.

Введите значение p-value в поле ниже с точностью до трех знаков в десятичной части. Используйте точку для разделения целой и дробной части.


In [11]:
def calculate_strat_mean(data, strata_name, gen_pop_weights, target_value):
    """Вычисляет стратифицированное среднее. использует веса из генеральной популяции (передаются как словарь)"""
    strats_means = data.groupby(strata_name)[target_value].mean()

    data_means_weights = pd.merge(
        pd.Series(strats_means, name="value_means"),
        pd.Series(gen_pop_weights, name="weight"),
        how="inner",
        left_index=True,
        right_index=True,
    )

    mean_strat = (
        data_means_weights["weight"] * data_means_weights["value_means"]
    ).sum()
    return mean_strat


def get_strat_ttest(df_A, df_B, strata_name, target_value, gen_pop_weights):
    """Проверяет гипотезу о равенстве средних для стратифицированного среднего и стратифицировнной дисперсии.

    return - pvalue.
    """
    mean_strat_A = calculate_strat_mean(
        df_A, strata_name, gen_pop_weights, target_value
    )
    mean_strat_B = calculate_strat_mean(
        df_B, strata_name, gen_pop_weights, target_value
    )

    var_strat_A = calculate_strat_var(
        df_A, strata_name, gen_pop_weights, target_value
    )
    var_strat_B = calculate_strat_var(
        df_B, strata_name, gen_pop_weights, target_value
    )

    delta_mean_strat = mean_strat_A - mean_strat_B
    std_mean_strat = np.sqrt(var_strat_A / len(df_A) + var_strat_B / len(df_B))
    t_stat_strat = delta_mean_strat / std_mean_strat
    p_value = 2 * (1 - stats.norm.cdf(np.abs(t_stat_strat)))

    inference = {"t_stat": t_stat_strat, "p_value": p_value}
    return inference

In [12]:
strat_pval_rs = get_strat_ttest(
    group_a,
    group_b,
    "is_capital",
    "ARPU",
    {"capital": capital_wght, "region": (1 - capital_wght)},
)["p_value"]

In [13]:
round(strat_pval_rs, 3)

np.float64(0.018)

### Задание 4 (1/2)
Возьмите за основу данные по контрольной группе (group = 0) и рассчитайте необходимый размер выборки, чтобы найти абсолютный MDE, равный наблюдаемой разнице средних, между тестовой и контрольной группами.

Параметры эксперимента:

- `alpha` = 0.05
- `beta` = 0.2
- критерий двусторонний
- выборка 50/50

Каким будет размер группы для простого t-test?

Введите ответ в поле ниже в формате целого числа.

In [14]:
abs_MDE = group_b["ARPU"].mean() - group_a["ARPU"].mean()
basic_cohen_D_effect_size = abs_MDE / group_a["ARPU"].std()

basic_sample_size = int(
    tt_ind_solve_power(
        effect_size=basic_cohen_D_effect_size,
        alpha=0.05,
        power=0.8,
        nobs1=None,
        ratio=1,
    )
)

In [15]:
basic_sample_size

11368

### Задание 4 (2/2)
Каким будет размер группы для стратифицированного t-test?

Введите ответ в поле ниже в формате целого числа.

In [16]:
capital_wght = len(group_a[group_a["is_capital"] == "capital"]) / len(group_a)

strat_std = (
    calculate_strat_var(
        group_a,
        "is_capital",
        {"capital": capital_wght, "region": (1 - capital_wght)},
        "ARPU",
    )
    ** 0.5
)

strat_cohen_D_effect_size = abs_MDE / strat_std

strat_sample_size = int(
    tt_ind_solve_power(
        effect_size=strat_cohen_D_effect_size,
        alpha=0.05,
        power=0.8,
        nobs1=None,
        ratio=1,
    )
)

In [17]:
strat_sample_size

8660